In [17]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder

In [18]:
dataset = pd.read_csv('CKD.csv')

In [19]:
categorical_cols=dataset.select_dtypes('object','category').columns #to find the columns that have categorical data in it.
categorical_cols

Index(['sg', 'rbc', 'pc', 'pcc', 'ba', 'htn', 'dm', 'cad', 'appet', 'pe',
       'ane', 'classification'],
      dtype='object')

In [20]:
for col in categorical_cols:
    print(dataset[col].value_counts())
    print("-"*30)

sg
a    152
c     84
b     81
d     75
e      7
Name: count, dtype: int64
------------------------------
rbc
normal      352
abnormal     47
Name: count, dtype: int64
------------------------------
pc
normal      323
abnormal     76
Name: count, dtype: int64
------------------------------
pcc
notpresent    357
present        42
Name: count, dtype: int64
------------------------------
ba
notpresent    377
present        22
Name: count, dtype: int64
------------------------------
htn
no     253
yes    146
Name: count, dtype: int64
------------------------------
dm
no     263
yes    136
Name: count, dtype: int64
------------------------------
cad
no     365
yes     34
Name: count, dtype: int64
------------------------------
appet
yes     316
poor     83
Name: count, dtype: int64
------------------------------
pe
poor    322
yes      77
Name: count, dtype: int64
------------------------------
ane
no     339
yes     60
Name: count, dtype: int64
------------------------------
classification


In [21]:
dataset.columns

Index(['age', 'bp', 'sg', 'al', 'su', 'rbc', 'pc', 'pcc', 'ba', 'bgr', 'bu',
       'sc', 'sod', 'pot', 'hrmo', 'pcv', 'wc', 'rc', 'htn', 'dm', 'cad',
       'appet', 'pe', 'ane', 'classification'],
      dtype='object')

In [22]:
indep=dataset[['age', 'bp', 'sg', 'al', 'su', 'rbc', 'pc', 'pcc', 'ba', 'bgr', 'bu','sc', 'sod', 'pot', 'hrmo', 'pcv', 'wc', 'rc', 'htn', 'dm', 'cad',
       'appet', 'pe', 'ane']]

In [23]:
dep=dataset[['classification']]

In [24]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(indep,dep,test_size=0.30,random_state=0)

In [25]:
'''categories_order = [
    ['a','b','c','d','e'],
    ['normal', 'abnormal'], 
    ['normal', 'abnormal'], 
    ['notpresent','present'],
    ['notpresent','present'],
    ['no','yes'],
    ['no','yes'],
    ['no','yes'],
    ['yes','poor'],
    ['poor','yes'],
    ['no','yes'],
    ['yes','no']
]
oe = OrdinalEncoder(categories=categories_order)
columns_to_encode = ['sg', 'rbc', 'pc', 'pcc', 'ba', 'htn', 'dm', 'cad', 'appet', 'pe','ane', 'classification']
dataset[columns_to_encode] = oe.fit_transform(dataset[columns_to_encode])
'''

"categories_order = [\n    ['a','b','c','d','e'],\n    ['normal', 'abnormal'], \n    ['normal', 'abnormal'], \n    ['notpresent','present'],\n    ['notpresent','present'],\n    ['no','yes'],\n    ['no','yes'],\n    ['no','yes'],\n    ['yes','poor'],\n    ['poor','yes'],\n    ['no','yes'],\n    ['yes','no']\n]\noe = OrdinalEncoder(categories=categories_order)\ncolumns_to_encode = ['sg', 'rbc', 'pc', 'pcc', 'ba', 'htn', 'dm', 'cad', 'appet', 'pe','ane', 'classification']\ndataset[columns_to_encode] = oe.fit_transform(dataset[columns_to_encode])\n"

In [26]:
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=0)
x_train = enc.fit_transform(x_train)
x_test = enc.transform(x_test)

ValueError: The used value for unknown_value 0 is one of the values already used for encoding the seen categories.

In [10]:
from sklearn.naive_bayes import CategoricalNB

classifier = CategoricalNB()

from sklearn.model_selection import GridSearchCV
catnb_param_grid = {
       'alpha': [0.001, 0.01, 0.1],    
       'fit_prior': [True, False]
}
grid=GridSearchCV(classifier,catnb_param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid.fit(x_train,y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


C:\Users\huawei\anaconda3\Lib\site-packages\sklearn\model_selection\_search.py:1108: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan]
  warnings.warn(
C:\Users\huawei\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=CategoricalNB(), n_jobs=-1,
             param_grid={'alpha': [0.001, 0.01, 0.1],
                         'fit_prior': [True, False]},
             scoring='f1_weighted', verbose=3)

In [11]:
print("The best parameter:",grid.best_params_)

The best parameter: {'alpha': 0.001, 'fit_prior': True}


In [12]:
y_pred = grid.predict(x_test)

ValueError: Negative values in data passed to CategoricalNB (input X).

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, y_pred)              
print(clf_report)
print(cm)

In [ ]:
from sklearn.metrics import f1_score
f1_wght_average=f1_score(y_test,y_pred,average='weighted')
print("The F1 score final:",f1_wght_average)

In [ ]:
from sklearn.metrics import roc_auc_score
scr=roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])
print("The roc_auc_score is:",scr)